In [36]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

In [61]:
import numpy as np

### Data Acquisition 

In [37]:
df = pd.read_csv('C:/Users/Abhigyan/Downloads/Cerina Healthcare/data_sources.csv', encoding='ISO-8859-1')


In [3]:
df

,Paper,Authors,Year,Platform,Target Outcomes,Labeling Methodology,Size,Availability,Additional Comments,Dataset Link (if any),Reference Link
0,CLPsych 2015 Shared Task: Depression and PTSD ...,"Coppersmith, Dredze, Harman, Hollingshead, Mit...",2015,Twitter,"Depression, PTSD, Control","Regular-expressions (e.g. ""I was just diagnose...","Train (326 depressed, 246 PTSD, 573 control); ...",Available via Signed Agreement,NaN,NaN,https://www.aclweb.org/anthology/W15-1204/
1,Quantifying Mental Health Signals in Twitter,"Coppersmith, Dredze, Harman",2014,Twitter,"Bipolar Disorder, Depression, PTSD, Seasonal A...","Regular-expressions (e.g. ""I was just diagnose...",Bipolar: 394 individuals (992k tweets)\nDepres...,Available via Signed Agreement,NaN,NaN,https://www.aclweb.org/anthology/W14-3207/
2,From ADHD to SAD: Analyzing the Language of Me...,"Coppersmith, Dredze, Harman, Hollingshead",2015,Twitter,"ADHD, Anxiety, Bipolar Disorder, Borderline Pe...","Regular-expressions (e.g. ""I was just diagnose...",ADHD: 102 individuals (384k tweets)\nAnxiety: ...,Available via Signed Agreement,NaN,NaN,https://www.aclweb.org/anthology/W15-1201/
3,Towards Augmenting Crisis Counselor Training b...,"DeMasi, Hearst, Recht",2019,Synthetic Crisis Text Conversations,None (Message Retrieval Task),Trained counselor role-play,"253 conversations; 9,062 visitor messages; 5,3...",NaN,NaN,NaN,https://www.aclweb.org/anthology/W19-3001/
4,Characterizing and Predicting Postpartum Depre...,"De Choudhury, Counts, Horvitz, Hoff",2014,Facebook,Post Partum Depression,PHQ-9 survey with confirmed clinical diagnoses,"165 individuals (137 w/o PPD, 28 w/ PPD); 578,...",NaN,NaN,NaN,http://www.munmund.net/pubs/cscw_14_1.pdf
...,...,...,...,...,...,...,...,...,...,...,...
135,"""Let Me Tell You About Your Mental Health!"" Co...","Gaur, Kursuncu, Alambo, Sheth, Daniulaityle, T...",2018,Reddit,"Anxiety, Borderline Personality, Bipolar, Opia...",Subreddit participation,All individuals who posted in manually identif...,Reproducible via API,From Gkotsis et al. (2016),NaN,https://dl.acm.org/doi/abs/10.1145/3269206.327...
136,Identifying Depressive Users in Twitter Using ...,"Kang, Yoon, Yi Kim",2019,Twitter,Depression,CES-D,45 individuals,Not Available,Korean Language Only,NaN,https://info.computer.org/csdl/proceedings-art...
137,Inferring Social Media Users' Mental Health St...,"Xu, Pérez-Rosas, Mihalcea",2020,Flickr,Mental Health (General),Tag-based proxy labelling,"Mental Illness (770 users, 14,781 posts); Pre-...",Reproducible via API,NaN,NaN,https://www.aclweb.org/anthology/2020.lrec-1.772/
138,eRISK 2017: CLEF Lab on Early Risk Prediction ...,"Losada, Crestani, Parapar",2017,Reddit,Depression,Regular expressions used to identify individua...,"Depression (135 users, 49,557 posts); Control ...",Available via Signed Agreement,Dataset originally proposed in Losada et al. (...,NaN,https://link.springer.com/chapter/10.1007/978-...


### Data Preprocessing

In [4]:
df.describe()

,Year
count,140.000000
mean,2016.650000
std,1.888721
min,2012.000000
25%,2015.000000
50%,2017.000000
75%,2018.000000
max,2020.000000


In [5]:
df.columns

Index(['Paper', 'Authors', 'Year', 'Platform', 'Target Outcomes',
       'Labeling Methodology', 'Size', 'Availability', 'Additional Comments',
       'Dataset Link (if any)', 'Reference Link'],
      dtype='object')

In [6]:
df.shape

(140, 11)

In [38]:
df.isna().sum()

Paper                      0
Authors                    0
Year                       0
Platform                   0
Target Outcomes            0
Labeling Methodology       0
Size                       0
Availability              64
Additional Comments       98
Dataset Link (if any)    128
Reference Link             0
dtype: int64

In [39]:
df= df.dropna()

In [40]:
for i in df.columns:
    print(i, ':', df[i].nunique())

Paper : 2
Authors : 2
Year : 2
Platform : 2
Target Outcomes : 2
Labeling Methodology : 2
Size : 2
Availability : 1
Additional Comments : 2
Dataset Link (if any) : 2
Reference Link : 2


In [41]:
df.dtypes

Paper                    object
Authors                  object
Year                      int64
Platform                 object
Target Outcomes          object
Labeling Methodology     object
Size                     object
Availability             object
Additional Comments      object
Dataset Link (if any)    object
Reference Link           object
dtype: object

In [12]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Abhigyan\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [13]:
import nltk
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to
[nltk_data]    |     C:\Users\Abhigyan\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping corpora\abc.zip.
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     C:\Users\Abhigyan\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping corpora\alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     C:\Users\Abhigyan\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping taggers\averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     C:\Users\Abhigyan\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers\averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     C:\Users\Abhigyan\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping grammars\basque_grammars.zip.

True

In [14]:
stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()

In [42]:
import spacy

In [43]:
nlp = spacy.load("en_core_web_sm")

In [44]:
def preprocess_text(text):
    doc = nlp(text)
    tokens = [token.lemma_ for token in doc if not token.is_stop and token.is_alpha]
    return ' '.join(tokens)

In [45]:
df['preprocessed_text'] = df['Target Outcomes'].apply(preprocess_text)

### Feature extraction

In [46]:
tfidf_vectorizer = TfidfVectorizer()

In [47]:
X = tfidf_vectorizer.fit_transform(df['preprocessed_text'])

In [48]:
y = df['Target Outcomes']

In [57]:
# Check the distribution of labels
label_distribution = df['Target Outcomes'].value_counts()
print(label_distribution)


Stress, Stress (Stressor and Stress Subject)    1
Aggression                                      1
Name: Target Outcomes, dtype: int64


In [59]:
from sklearn.preprocessing import LabelEncoder

# Convert labels to integers using label encoding
label_encoder = LabelEncoder()
df['labels_encoded'] = label_encoder.fit_transform(df['Target Outcomes'])


In [69]:
df['Target Outcomes']

115    Stress, Stress (Stressor and Stress Subject)
121                                      Aggression
Name: Target Outcomes, dtype: object

In [71]:
df[['labels_encoded']].dtypes

labels_encoded    int32
dtype: object

In [49]:
pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [53]:
from sklearn.model_selection import train_test_split


In [54]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [66]:

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error

In [72]:
df.dtypes

Paper                    object
Authors                  object
Year                      int64
Platform                 object
Target Outcomes          object
Labeling Methodology     object
Size                     object
Availability             object
Additional Comments      object
Dataset Link (if any)    object
Reference Link           object
preprocessed_text        object
labels_encoded            int32
dtype: object

In [67]:
lr_model = LinearRegression() #initiallizing the linear regression model

In [68]:
lr_model.fit(X_train, y_train) #Fitting the Linear regression model on the X_train and y_train

ValueError: could not convert string to float: 'Stress, Stress (Stressor and Stress Subject)'